In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
date = "2022-6"

my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"
# This is my key. I'm nice and I have it posted. If you will be doing more with this
# please get your own key!

In [3]:
end_use = "naics?get=CON_VAL_MO,CTY_CODE,CTY_NAME,SUMMARY_LVL"

url = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 
url = url + my_key + "&time==from+2013-01"

r = requests.get(url) 
    
print(r)
    
df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

df.columns = r.json()[0]

df["total_imports"] = df["CON_VAL_MO"].astype(float)

df = df[df.SUMMARY_LVL == "DET"]

grp = df.groupby(["CTY_NAME"])

top_products = grp.agg({"total_imports":"sum","CTY_CODE":"first"})

country_list = list(top_products.sort_values(by = "total_imports", ascending = False).CTY_CODE)[0:31]


['TOTAL FOR ALL COUNTRIES','NAFTA','EUROPEAN UNION']

<Response [200]>


['TOTAL FOR ALL COUNTRIES', 'NAFTA', 'EUROPEAN UNION']

In [4]:
country_list[0] = ""

In [5]:
country_list

['',
 '5700',
 '2010',
 '1220',
 '5880',
 '4280',
 '5800',
 '5520',
 '4120',
 '5330',
 '4190',
 '5830',
 '4759',
 '4279',
 '4419',
 '5570',
 '5490',
 '3510',
 '5170',
 '5590',
 '4210',
 '5081',
 '5600',
 '4621',
 '4231',
 '4700',
 '3010',
 '4330',
 '5650',
 '4010',
 '6021']

In [8]:
end_use = "hs?get=CTY_NAME,CON_VAL_MO,CAL_DUT_MO,I_COMMODITY,I_COMMODITY_SDESC"

surl = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 

surl  = surl + my_key + "&time=" + "from+2013-01" + "&COMM_LVL=HS2" 

for xxx in country_list:
    
    out_file = ".\\data"+ "\\imports\\" + xxx + "data-" + date + ".parquet"
    
    if xxx == "":
        out_file = ".\\data"+ "\\imports\\" + "TOTAL" + "data-" + date + ".parquet"
    
    
    if os.path.exists(out_file):
        
        print("Already have downloaded file")
        
        continue
    
    print(xxx)
    
    url = surl + "&CTY_CODE=" + xxx
    
    if xxx == "":
        url = surl
    
    r = requests.get(url) 
    
    print(r)
    
    foo = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

    foo.columns = r.json()[0]

    pq.write_table(pa.Table.from_pandas(foo), out_file)

Already have downloaded file
5700
<Response [200]>
2010
<Response [200]>
1220
<Response [200]>
5880
<Response [200]>
4280
<Response [200]>
5800
<Response [200]>
5520
<Response [200]>
4120
<Response [200]>
5330
<Response [200]>
4190
<Response [200]>
5830
<Response [200]>
4759
<Response [200]>
4279
<Response [200]>
4419
<Response [200]>
5570
<Response [200]>
5490
<Response [200]>
3510
<Response [200]>
5170
<Response [200]>
5590
<Response [200]>
4210
<Response [200]>
5081
<Response [200]>
5600
<Response [200]>
4621
<Response [200]>
4231
<Response [200]>
4700
<Response [200]>
3010
<Response [200]>
4330
<Response [200]>
5650
<Response [200]>
4010
<Response [200]>
6021
<Response [200]>


In [9]:
country_list[0] = "TOTAL"

df = pd.DataFrame([])
#this is for all countries...

for xxx in country_list:
    
    file = ".\\data\\imports\\" + xxx + "data-" + date + ".parquet"

    foo = pq.read_table(file).to_pandas()
    
    #recentfile = ".\\data\\imports\\" + xxx + "-" + date + "-" + "data.parquet"
    
    #foo = foo.append(pq.read_table(recentfile).to_pandas())
    
    #pq.write_table(pa.Table.from_pandas(foo), file)
    #this combines the two periods
    
    df = df.append(foo)
            
df.time = pd.to_datetime(df.time, format="%Y-%m")

df["imports"] = df["CON_VAL_MO"].astype(float)

df["duty"] = df["CAL_DUT_MO"].astype(float)

total = df.groupby(["CTY_NAME", "time"]).agg({"imports": "sum", "duty": "sum"})
                   
total.reset_index(inplace = True)
                   
total["I_COMMODITY"] = "00"
total["I_COMMODITY_SDESC"] = "ALL PRODUCTS"
                   
df = df.append(total)

df["itariff"] = 100*df["duty"]/df["imports"]

df.drop(["CON_VAL_MO", "COMM_LVL", "CTY_CODE", "CAL_DUT_MO"], axis = 1, inplace = True)

df["I_COMMODITY_SDESC"] = "HS CODE " + df["I_COMMODITY"] + ", " + df["I_COMMODITY_SDESC"]

df["I_COMMODITY_SDESC"].replace({"HS CODE 00, ALL PRODUCTS": "ALL PRODUCTS"}, inplace = True)

df.set_index(["CTY_NAME","I_COMMODITY_SDESC","time"],inplace = True)

out_file = ".\\data"+ "\\top30-HS2-imports.parquet"

pq.write_table(pa.Table.from_pandas(df), out_file)

In [10]:
df.loc["CHINA","ALL PRODUCTS"]["2022"]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  """Entry point for launching an IPython kernel.


,I_COMMODITY,imports,duty,itariff
time,,,,
2022-01-01,00,4.647180e+10,5.538087e+09,11.917094
2022-02-01,00,4.199971e+10,5.009983e+09,11.928613
2022-03-01,00,4.697708e+10,5.426281e+09,11.550913
2022-04-01,00,4.064364e+10,4.876375e+09,11.997879
2022-05-01,00,4.223204e+10,4.962144e+09,11.749712
2022-06-01,00,4.892717e+10,5.138167e+09,10.501664


In [11]:
file = ".\\data\\imports\\" + "TOTAL" + "data.parquet"

foo = pq.read_table(file).to_pandas()
    
foo

,CTY_NAME,CON_VAL_MO,CAL_DUT_MO,I_COMMODITY,I_COMMODITY_SDESC,time,COMM_LVL
0,TOTAL FOR ALL COUNTRIES,784536648,4504327,07,EDIBLE VEGETABLES & CERTAIN ROOTS & TUBERS,2013-01,HS2
1,TOTAL FOR ALL COUNTRIES,315892605,2650541,17,SUGARS AND SUGAR CONFECTIONARY,2013-01,HS2
2,TOTAL FOR ALL COUNTRIES,334605750,723046,10,CEREALS,2013-01,HS2
3,TOTAL FOR ALL COUNTRIES,144113171,409478,13,"LAC; GUMS, RESINS & OTHER VEGETABLE SAP & EXTRACT",2013-01,HS2
4,TOTAL FOR ALL COUNTRIES,371169570,2283401,18,COCOA AND COCOA PREPARATIONS,2013-01,HS2
...,...,...,...,...,...,...,...
9991,TOTAL FOR ALL COUNTRIES,794906854,2374814,09,"COFFEE, TEA, MATE & SPICES",2021-06,HS2
9992,TOTAL FOR ALL COUNTRIES,146629956,1172022,10,CEREALS,2021-06,HS2
9993,TOTAL FOR ALL COUNTRIES,186372088,2220530,11,MILLING PRODUCTS; MALT; STARCH; INULIN; WHT GL...,2021-06,HS2
9994,TOTAL FOR ALL COUNTRIES,340184476,3851904,12,"OIL SEEDS ETC.; MISC GRAIN, SEED, FRUIT, PLANT...",2021-06,HS2


In [12]:
foo

,CTY_NAME,CON_VAL_MO,CAL_DUT_MO,I_COMMODITY,I_COMMODITY_SDESC,time,COMM_LVL
0,TOTAL FOR ALL COUNTRIES,784536648,4504327,07,EDIBLE VEGETABLES & CERTAIN ROOTS & TUBERS,2013-01,HS2
1,TOTAL FOR ALL COUNTRIES,315892605,2650541,17,SUGARS AND SUGAR CONFECTIONARY,2013-01,HS2
2,TOTAL FOR ALL COUNTRIES,334605750,723046,10,CEREALS,2013-01,HS2
3,TOTAL FOR ALL COUNTRIES,144113171,409478,13,"LAC; GUMS, RESINS & OTHER VEGETABLE SAP & EXTRACT",2013-01,HS2
4,TOTAL FOR ALL COUNTRIES,371169570,2283401,18,COCOA AND COCOA PREPARATIONS,2013-01,HS2
...,...,...,...,...,...,...,...
9991,TOTAL FOR ALL COUNTRIES,794906854,2374814,09,"COFFEE, TEA, MATE & SPICES",2021-06,HS2
9992,TOTAL FOR ALL COUNTRIES,146629956,1172022,10,CEREALS,2021-06,HS2
9993,TOTAL FOR ALL COUNTRIES,186372088,2220530,11,MILLING PRODUCTS; MALT; STARCH; INULIN; WHT GL...,2021-06,HS2
9994,TOTAL FOR ALL COUNTRIES,340184476,3851904,12,"OIL SEEDS ETC.; MISC GRAIN, SEED, FRUIT, PLANT...",2021-06,HS2


In [13]:
foo = pq.read_table(".\\data\\top30-HS2-imports.parquet").to_pandas()

In [14]:
foo.tail()

I_COMMODITY       imports         duty  \
CTY_NAME I_COMMODITY_SDESC time                                                
VIETNAM  ALL PRODUCTS      2022-02-01          00  8.100665e+09  425714208.0   
                           2022-03-01          00  1.119875e+10  561136237.0   
                           2022-04-01          00  1.103069e+10  537848699.0   
                           2022-05-01          00  1.142300e+10  551582339.0   
                           2022-06-01          00  1.123672e+10  501210110.0   

                                        itariff  
CTY_NAME I_COMMODITY_SDESC time                  
VIETNAM  ALL PRODUCTS      2022-02-01  5.255300  
                           2022-03-01  5.010702  
                           2022-04-01  4.875930  
                           2022-05-01  4.828699  
                           2022-06-01  4.460464

In [15]:
foo.loc["TOTAL FOR ALL COUNTRIES"]

I_COMMODITY  \
I_COMMODITY_SDESC                                  time                     
HS CODE 07, EDIBLE VEGETABLES & CERTAIN ROOTS &... 2013-01-01          07   
HS CODE 17, SUGARS AND SUGAR CONFECTIONARY         2013-01-01          17   
HS CODE 10, CEREALS                                2013-01-01          10   
HS CODE 13, LAC; GUMS, RESINS & OTHER VEGETABLE... 2013-01-01          13   
HS CODE 18, COCOA AND COCOA PREPARATIONS           2013-01-01          18   
...                                                                   ...   
ALL PRODUCTS                                       2022-02-01          00   
                                                   2022-03-01          00   
                                                   2022-04-01          00   
                                                   2022-05-01          00   
                                                   2022-06-01          00   

                                                                    imports  \
I_COMMODITY_SDESC                                  time                       
HS CODE 07, EDIBLE VEGETABLES & CERTAIN ROOTS &... 2013-01-01  7.845366e+08   
HS CODE 17, SUGARS AND SUGAR CONFECTIONARY         2013-01-01  3.158926e+08   
HS CODE 10, CEREALS                                2013-01-01  3.346058e+08   
HS CODE 13, LAC; GUMS, RESINS & OTHER VEGETABLE... 2013-01-01  1.441132e+08   
HS CODE 18, COCOA AND COCOA PREPARATIONS           2013-01-01  3.711696e+08   
...                                                                     ...   
ALL PRODUCTS                                       2022-02-01  2.329552e+11   
                                                   2022-03-01  2.963489e+11   
                                                   2022-04-01  2.715095e+11   
                                                   2022-05-01  2.815207e+11   
                                                   2022-06-01  2.865001e+11   

                                                                       duty  \
I_COMMODITY_SDESC                                  time                       
HS CODE 07, EDIBLE VEGETABLES & CERTAIN ROOTS &... 2013-01-01  4.504327e+06   
HS CODE 17, SUGARS AND SUGAR CONFECTIONARY         2013-01-01  2.650541e+06   
HS CODE 10, CEREALS                                2013-01-01  7.230460e+05   
HS CODE 13, LAC; GUMS, RESINS & OTHER VEGETABLE... 2013-01-01  4.094780e+05   
HS CODE 18, COCOA AND COCOA PREPARATIONS           2013-01-01  2.283401e+06   
...                                                                     ...   
ALL PRODUCTS                                       2022-02-01  7.314144e+09   
                                                   2022-03-01  8.411181e+09   
                                                   2022-04-01  7.686908e+09   
                                                   2022-05-01  7.883407e+09   
                                                   2022-06-01  7.948780e+09   

                                                                itariff  
I_COMMODITY_SDESC                                  time                  
HS CODE 07, EDIBLE VEGETABLES & CERTAIN ROOTS &... 2013-01-01  0.574139  
HS CODE 17, SUGARS AND SUGAR CONFECTIONARY         2013-01-01  0.839064  
HS CODE 10, CEREALS                                2013-01-01  0.216089  
HS CODE 13, LAC; GUMS, RESINS & OTHER VEGETABLE... 2013-01-01  0.284136  
HS CODE 18, COCOA AND COCOA PREPARATIONS           2013-01-01  0.615191  
...                                                                 ...  
ALL PRODUCTS                                       2022-02-01  3.139721  
                                                   2022-03-01  2.838270  
                                                   2022-04-01  2.831175  
                                                   2022-05-01  2.800294  
                                                   2022-06-01  2.774442  

[11286 rows x 4 columns]